In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.backends import cudnn
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
import os
from model_pytorch import PyramidNet
from torchsummary import summary
from torch.optim import lr_scheduler

device = 'cuda:1'
check_point = False
best_acc = 0
start_epoch = 0
lr = 0.01

In [2]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4464), (0.2023, 0.1994, 0.2010)),])
transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4464), (0.2023, 0.1994, 0.2010)),])

trainset = CIFAR10('./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
testset = CIFAR10('./data', train=False, download=False, transform=transform_test)
testloader = DataLoader(testset, batch_size=10, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [3]:
# Model
net = PyramidNet(depth=272, alpha=200)
# net = net.to('cuda')
# summary(net, (3, 32, 32), device='cuda')
net = net.to(device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 18, 32, 32]             288
       BatchNorm2d-5           [-1, 18, 32, 32]              36
            Conv2d-6           [-1, 18, 32, 32]           2,916
       BatchNorm2d-7           [-1, 18, 32, 32]              36
            Conv2d-8           [-1, 72, 32, 32]           1,296
       BatchNorm2d-9           [-1, 72, 32, 32]             144
  BottleneckBlock-10           [-1, 72, 32, 32]               0
      BatchNorm2d-11           [-1, 72, 32, 32]             144
           Conv2d-12           [-1, 20, 32, 32]           1,440
      BatchNorm2d-13           [-1, 20, 32, 32]              40
           Conv2d-14           [-1, 20,

In [4]:
cudnn.benchmark = True

if check_point:
    print("Resuming CheckPoint")
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
decay_epoch = [75, 150]
step_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)

# Training
def train(epoch):
    print("\nEpoch: %d" % epoch)
    net.train()
    train_loss = 0
    correct = 0 
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print("Loss: %.3f | Acc %.3f%% (%d/%d)" %(train_loss/(batch_idx+1), 100.*correct/total, correct, total))


# Test
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)' %(test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    acc = 100.*correct/total
    if acc > best_acc:
        print("Saving")
        state = {'net':net.state_dict(), 'acc':acc, 'epoch':epoch,}
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)


Epoch: 0
Loss: 2.350 | Acc 12.500% (2/16)
Loss: 2.338 | Acc 9.375% (3/32)
Loss: 2.331 | Acc 6.250% (3/48)
Loss: 2.318 | Acc 7.812% (5/64)
Loss: 2.318 | Acc 7.500% (6/80)
Loss: 2.320 | Acc 7.292% (7/96)
Loss: 2.321 | Acc 8.036% (9/112)
Loss: 2.312 | Acc 9.375% (12/128)
Loss: 2.316 | Acc 8.333% (12/144)
Loss: 2.329 | Acc 8.750% (14/160)
Loss: 2.323 | Acc 8.523% (15/176)
Loss: 2.326 | Acc 8.333% (16/192)
Loss: 2.325 | Acc 9.135% (19/208)
Loss: 2.315 | Acc 9.821% (22/224)
Loss: 2.316 | Acc 10.000% (24/240)
Loss: 2.317 | Acc 10.156% (26/256)
Loss: 2.320 | Acc 9.926% (27/272)
Loss: 2.322 | Acc 9.722% (28/288)
Loss: 2.321 | Acc 9.539% (29/304)
Loss: 2.324 | Acc 9.375% (30/320)
Loss: 2.321 | Acc 9.226% (31/336)
Loss: 2.317 | Acc 9.659% (34/352)
Loss: 2.320 | Acc 10.054% (37/368)
Loss: 2.314 | Acc 10.417% (40/384)
Loss: 2.318 | Acc 10.250% (41/400)
Loss: 2.309 | Acc 10.337% (43/416)
Loss: 2.307 | Acc 10.880% (47/432)
Loss: 2.303 | Acc 10.938% (49/448)
Loss: 2.308 | Acc 10.560% (49/464)
Loss: 2

Loss: 2.130 | Acc 19.328% (702/3632)
Loss: 2.129 | Acc 19.408% (708/3648)
Loss: 2.129 | Acc 19.378% (710/3664)
Loss: 2.130 | Acc 19.375% (713/3680)
Loss: 2.129 | Acc 19.426% (718/3696)
Loss: 2.128 | Acc 19.450% (722/3712)
Loss: 2.126 | Acc 19.474% (726/3728)
Loss: 2.125 | Acc 19.498% (730/3744)
Loss: 2.124 | Acc 19.574% (736/3760)
Loss: 2.123 | Acc 19.597% (740/3776)
Loss: 2.122 | Acc 19.699% (747/3792)
Loss: 2.121 | Acc 19.722% (751/3808)
Loss: 2.122 | Acc 19.718% (754/3824)
Loss: 2.122 | Acc 19.661% (755/3840)
Loss: 2.123 | Acc 19.658% (758/3856)
Loss: 2.123 | Acc 19.680% (762/3872)
Loss: 2.122 | Acc 19.624% (763/3888)
Loss: 2.122 | Acc 19.647% (767/3904)
Loss: 2.122 | Acc 19.668% (771/3920)
Loss: 2.121 | Acc 19.665% (774/3936)
Loss: 2.120 | Acc 19.661% (777/3952)
Loss: 2.120 | Acc 19.682% (781/3968)
Loss: 2.118 | Acc 19.729% (786/3984)
Loss: 2.117 | Acc 19.750% (790/4000)
Loss: 2.117 | Acc 19.771% (794/4016)
Loss: 2.116 | Acc 19.767% (797/4032)
Loss: 2.115 | Acc 19.837% (803/4048)
L

Loss: 2.027 | Acc 23.596% (1680/7120)
Loss: 2.026 | Acc 23.641% (1687/7136)
Loss: 2.026 | Acc 23.658% (1692/7152)
Loss: 2.025 | Acc 23.675% (1697/7168)
Loss: 2.025 | Acc 23.705% (1703/7184)
Loss: 2.025 | Acc 23.694% (1706/7200)
Loss: 2.025 | Acc 23.683% (1709/7216)
Loss: 2.025 | Acc 23.700% (1714/7232)
Loss: 2.025 | Acc 23.689% (1717/7248)
Loss: 2.025 | Acc 23.692% (1721/7264)
Loss: 2.024 | Acc 23.723% (1727/7280)
Loss: 2.024 | Acc 23.725% (1731/7296)
Loss: 2.024 | Acc 23.742% (1736/7312)
Loss: 2.023 | Acc 23.758% (1741/7328)
Loss: 2.023 | Acc 23.775% (1746/7344)
Loss: 2.022 | Acc 23.845% (1755/7360)
Loss: 2.021 | Acc 23.875% (1761/7376)
Loss: 2.021 | Acc 23.891% (1766/7392)
Loss: 2.021 | Acc 23.907% (1771/7408)
Loss: 2.021 | Acc 23.895% (1774/7424)
Loss: 2.021 | Acc 23.925% (1780/7440)
Loss: 2.021 | Acc 23.900% (1782/7456)
Loss: 2.021 | Acc 23.889% (1785/7472)
Loss: 2.021 | Acc 23.851% (1786/7488)
Loss: 2.020 | Acc 23.867% (1791/7504)
Loss: 2.020 | Acc 23.896% (1797/7520)
Loss: 2.019 

Loss: 1.967 | Acc 26.089% (2755/10560)
Loss: 1.967 | Acc 26.068% (2757/10576)
Loss: 1.967 | Acc 26.086% (2763/10592)
Loss: 1.966 | Acc 26.112% (2770/10608)
Loss: 1.966 | Acc 26.082% (2771/10624)
Loss: 1.965 | Acc 26.090% (2776/10640)
Loss: 1.965 | Acc 26.126% (2784/10656)
Loss: 1.964 | Acc 26.153% (2791/10672)
Loss: 1.965 | Acc 26.132% (2793/10688)
Loss: 1.964 | Acc 26.168% (2801/10704)
Loss: 1.963 | Acc 26.175% (2806/10720)
Loss: 1.963 | Acc 26.164% (2809/10736)
Loss: 1.963 | Acc 26.181% (2815/10752)
Loss: 1.962 | Acc 26.189% (2820/10768)
Loss: 1.962 | Acc 26.196% (2825/10784)
Loss: 1.961 | Acc 26.241% (2834/10800)
Loss: 1.960 | Acc 26.267% (2841/10816)
Loss: 1.960 | Acc 26.274% (2846/10832)
Loss: 1.960 | Acc 26.281% (2851/10848)
Loss: 1.960 | Acc 26.298% (2857/10864)
Loss: 1.959 | Acc 26.314% (2863/10880)
Loss: 1.958 | Acc 26.349% (2871/10896)
Loss: 1.958 | Acc 26.338% (2874/10912)
Loss: 1.958 | Acc 26.336% (2878/10928)
Loss: 1.959 | Acc 26.352% (2884/10944)
Loss: 1.958 | Acc 26.378%

Loss: 1.911 | Acc 28.322% (3947/13936)
Loss: 1.910 | Acc 28.333% (3953/13952)
Loss: 1.910 | Acc 28.336% (3958/13968)
Loss: 1.909 | Acc 28.390% (3970/13984)
Loss: 1.909 | Acc 28.393% (3975/14000)
Loss: 1.908 | Acc 28.410% (3982/14016)
Loss: 1.908 | Acc 28.414% (3987/14032)
Loss: 1.908 | Acc 28.417% (3992/14048)
Loss: 1.908 | Acc 28.427% (3998/14064)
Loss: 1.908 | Acc 28.438% (4004/14080)
Loss: 1.907 | Acc 28.448% (4010/14096)
Loss: 1.906 | Acc 28.465% (4017/14112)
Loss: 1.906 | Acc 28.482% (4024/14128)
Loss: 1.906 | Acc 28.500% (4031/14144)
Loss: 1.906 | Acc 28.510% (4037/14160)
Loss: 1.906 | Acc 28.506% (4041/14176)
Loss: 1.905 | Acc 28.523% (4048/14192)
Loss: 1.905 | Acc 28.547% (4056/14208)
Loss: 1.905 | Acc 28.571% (4064/14224)
Loss: 1.904 | Acc 28.560% (4067/14240)
Loss: 1.904 | Acc 28.584% (4075/14256)
Loss: 1.903 | Acc 28.566% (4077/14272)
Loss: 1.903 | Acc 28.604% (4087/14288)
Loss: 1.903 | Acc 28.607% (4092/14304)
Loss: 1.902 | Acc 28.638% (4101/14320)
Loss: 1.902 | Acc 28.634%

...

Loss: 0.977 | Acc 66.181% (3272/4944)
Loss: 0.978 | Acc 66.129% (3280/4960)
Loss: 0.977 | Acc 66.158% (3292/4976)
Loss: 0.977 | Acc 66.186% (3304/4992)
Loss: 0.975 | Acc 66.254% (3318/5008)
Loss: 0.974 | Acc 66.282% (3330/5024)
Loss: 0.975 | Acc 66.210% (3337/5040)
Loss: 0.975 | Acc 66.258% (3350/5056)
Loss: 0.974 | Acc 66.285% (3362/5072)
Loss: 0.973 | Acc 66.293% (3373/5088)
Loss: 0.974 | Acc 66.281% (3383/5104)
Loss: 0.975 | Acc 66.211% (3390/5120)
Loss: 0.976 | Acc 66.141% (3397/5136)
Loss: 0.975 | Acc 66.130% (3407/5152)
Loss: 0.975 | Acc 66.157% (3419/5168)
Loss: 0.976 | Acc 66.107% (3427/5184)
Loss: 0.975 | Acc 66.173% (3441/5200)
Loss: 0.974 | Acc 66.219% (3454/5216)
Loss: 0.974 | Acc 66.246% (3466/5232)
Loss: 0.974 | Acc 66.273% (3478/5248)
Loss: 0.973 | Acc 66.242% (3487/5264)
Loss: 0.975 | Acc 66.231% (3497/5280)
Loss: 0.975 | Acc 66.220% (3507/5296)
Loss: 0.975 | Acc 66.246% (3519/5312)
Loss: 0.976 | Acc 66.216% (3528/5328)
Loss: 0.978 | Acc 66.168% (3536/5344)
Loss: 0.978 

Loss: 0.954 | Acc 67.190% (5644/8400)
Loss: 0.953 | Acc 67.193% (5655/8416)
Loss: 0.953 | Acc 67.208% (5667/8432)
Loss: 0.952 | Acc 67.223% (5679/8448)
Loss: 0.952 | Acc 67.238% (5691/8464)
Loss: 0.951 | Acc 67.229% (5701/8480)
Loss: 0.951 | Acc 67.255% (5714/8496)
Loss: 0.952 | Acc 67.223% (5722/8512)
Loss: 0.951 | Acc 67.249% (5735/8528)
Loss: 0.951 | Acc 67.240% (5745/8544)
Loss: 0.951 | Acc 67.255% (5757/8560)
Loss: 0.950 | Acc 67.269% (5769/8576)
Loss: 0.950 | Acc 67.249% (5778/8592)
Loss: 0.951 | Acc 67.240% (5788/8608)
Loss: 0.951 | Acc 67.254% (5800/8624)
Loss: 0.951 | Acc 67.245% (5810/8640)
Loss: 0.951 | Acc 67.248% (5821/8656)
Loss: 0.950 | Acc 67.239% (5831/8672)
Loss: 0.950 | Acc 67.265% (5844/8688)
Loss: 0.950 | Acc 67.279% (5856/8704)
Loss: 0.950 | Acc 67.282% (5867/8720)
Loss: 0.949 | Acc 67.285% (5878/8736)
Loss: 0.949 | Acc 67.299% (5890/8752)
Loss: 0.949 | Acc 67.324% (5903/8768)
Loss: 0.949 | Acc 67.338% (5915/8784)
Loss: 0.949 | Acc 67.341% (5926/8800)
Loss: 0.949 

Loss: 0.926 | Acc 68.225% (8056/11808)
Loss: 0.926 | Acc 68.234% (8068/11824)
Loss: 0.927 | Acc 68.218% (8077/11840)
Loss: 0.926 | Acc 68.244% (8091/11856)
Loss: 0.925 | Acc 68.270% (8105/11872)
Loss: 0.926 | Acc 68.262% (8115/11888)
Loss: 0.926 | Acc 68.263% (8126/11904)
Loss: 0.926 | Acc 68.263% (8137/11920)
Loss: 0.925 | Acc 68.281% (8150/11936)
Loss: 0.925 | Acc 68.265% (8159/11952)
Loss: 0.926 | Acc 68.257% (8169/11968)
Loss: 0.926 | Acc 68.233% (8177/11984)
Loss: 0.926 | Acc 68.217% (8186/12000)
Loss: 0.927 | Acc 68.217% (8197/12016)
Loss: 0.927 | Acc 68.218% (8208/12032)
Loss: 0.927 | Acc 68.194% (8216/12048)
Loss: 0.927 | Acc 68.203% (8228/12064)
Loss: 0.927 | Acc 68.220% (8241/12080)
Loss: 0.927 | Acc 68.196% (8249/12096)
Loss: 0.927 | Acc 68.205% (8261/12112)
Loss: 0.927 | Acc 68.206% (8272/12128)
Loss: 0.927 | Acc 68.231% (8286/12144)
Loss: 0.927 | Acc 68.240% (8298/12160)
Loss: 0.927 | Acc 68.216% (8306/12176)
Loss: 0.927 | Acc 68.184% (8313/12192)
Loss: 0.927 | Acc 68.185%

Loss: 0.914 | Acc 68.664% (10415/15168)
Loss: 0.913 | Acc 68.671% (10427/15184)
Loss: 0.914 | Acc 68.664% (10437/15200)
Loss: 0.914 | Acc 68.665% (10448/15216)
Loss: 0.915 | Acc 68.632% (10454/15232)
Loss: 0.915 | Acc 68.612% (10462/15248)
Loss: 0.915 | Acc 68.599% (10471/15264)
Loss: 0.916 | Acc 68.606% (10483/15280)
Loss: 0.916 | Acc 68.613% (10495/15296)
Loss: 0.917 | Acc 68.600% (10504/15312)
Loss: 0.916 | Acc 68.600% (10515/15328)
Loss: 0.917 | Acc 68.594% (10525/15344)
Loss: 0.916 | Acc 68.587% (10535/15360)
Loss: 0.917 | Acc 68.561% (10542/15376)
Loss: 0.917 | Acc 68.581% (10556/15392)
Loss: 0.917 | Acc 68.581% (10567/15408)
Loss: 0.917 | Acc 68.601% (10581/15424)
Loss: 0.917 | Acc 68.588% (10590/15440)
Loss: 0.917 | Acc 68.588% (10601/15456)
Loss: 0.916 | Acc 68.601% (10614/15472)
Loss: 0.916 | Acc 68.608% (10626/15488)
Loss: 0.916 | Acc 68.602% (10636/15504)
Loss: 0.916 | Acc 68.595% (10646/15520)
Loss: 0.916 | Acc 68.602% (10658/15536)
Loss: 0.916 | Acc 68.609% (10670/15552)


Loss: 0.909 | Acc 68.923% (12715/18448)
Loss: 0.909 | Acc 68.918% (12725/18464)
Loss: 0.909 | Acc 68.923% (12737/18480)
Loss: 0.909 | Acc 68.923% (12748/18496)
Loss: 0.909 | Acc 68.907% (12756/18512)
Loss: 0.909 | Acc 68.907% (12767/18528)
Loss: 0.909 | Acc 68.917% (12780/18544)
Loss: 0.908 | Acc 68.933% (12794/18560)
